In [20]:
import rasterio
import keras
import random
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Bidirectional
from keras.layers import Conv2D, MaxPooling2D, ConvLSTM2D, TimeDistributed, UpSampling2D, Concatenate, LSTM
from keras.layers import Activation, BatchNormalization
from keras.optimizers import SGD, Adadelta, Adam
from keras import Input
from keras import Model
import matplotlib as plt
import os
import sys
from sklearn.utils import class_weight
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities as util
import importlib
import rnn_tiles
import rnn_pixels
import numpy as np
importlib.reload(rnn_pixels)
importlib.reload(rnn_tiles)
importlib.reload(util)

<module 'utilities' from '/host/Desktop/cnn_dev/florence_mapping/utilities.py'>

In [21]:
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [22]:
lc_labels = rasterio.open('/deep_data/recurrent_data/NLCD_DATA/landcover/NLCD_2011_Land_Cover_L48_20190424.img')
canopy_labels = rasterio.open('/deep_data/recurrent_data/NLCD_DATA/canopy/CONUSCartographic_2_8_16/Cartographic/nlcd2011_usfs_conus_canopy_cartographic.img')

In [23]:
tiles = {}
landsat_datasets = {}
tiles['028012'] = ['20110324', '20110612', '20110831', '20111103']
tiles['029011'] = ['20110308', '20110425', '20110831', '20111103']
tiles['028011'] = ['20110308', '20110628', '20110831', '20111103']
for tile_number, dates in tiles.items():
    tile_datasets = []
    l8_image_paths = []
    for date in dates:
        l8_image_paths.append('/deep_data/recurrent_data/tile{}/combined/combined{}.tif'.format(tile_number, date))
    for fp in l8_image_paths:
        tile_datasets.append(rasterio.open(fp))
    landsat_datasets[tile_number] = tile_datasets

In [24]:
tile_size = 1
tile_list = ['028012', '029011', '028011']
class_count = 6
epochs = 200
batch_size = 25

# The next two cells are for making pixels and prepping them for training (balancing)
### This is unnecessary to run because I already saved them to text files so now you load them

In [ ]:
px = rnn_pixels.make_pixels(tile_size, tile_list)
px_to_use = px[:1000]
pixels = rnn_pixels.delete_bad_tiles(landsat_datasets,lc_labels, canopy_labels, px_to_use, tile_size)

In [ ]:
# gets balanced pixels locations 
w_tile_gen = rnn_tiles.rnn_tile_gen(landsat_datasets, lc_labels, canopy_labels, tile_size, class_count)
w_generator = w_tile_gen.tile_generator(pixels, batch_size=1, flatten=True, canopy=False)
total_labels = list()
count = 0
#buckets = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
buckets = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}

while count < len(pixels):
        image_b, label_b = next(w_generator)
        label_b = np.argmax(label_b)
        buckets[label_b].append(pixels[count]) # appends pixels to dictionary
        total_labels.append(label_b)
        count+=1
total_labels = np.asarray(total_labels)
total_labels = total_labels.reshape(len(total_labels)*tile_size*tile_size)
#weights_list = class_weight.compute_class_weight('balanced', np.unique(total_labels), total_labels)
count = 0 
for z, j in buckets.items():
    print(len(j))
    count += len(j)
print(count) 
use_px = []
use_px+=buckets[0][:3000]
use_px+=(buckets[1][:3000])
use_px+=(buckets[2][:3000])
use_px+=(buckets[3][:3000])
use_px+=(buckets[4][:3000])
use_px+=(buckets[5][:3000])
random.shuffle(use_px)
train_px, val_px, test_px = rnn_pixels.train_val_test_split(use_px, 0.7, 0.7)

# Load pixels

In [25]:
train_px, val_px, test_px = util.read_txt(['train_px.txt', 'val_px.txt', 'test_px.txt'])
train_px = rnn_pixels.delete_bad_tiles(landsat_datasets,lc_labels, canopy_labels, train_px, tile_size)
val_px =  rnn_pixels.delete_bad_tiles(landsat_datasets,lc_labels, canopy_labels, val_px, tile_size)
test_px =  rnn_pixels.delete_bad_tiles(landsat_datasets,lc_labels, canopy_labels, test_px, tile_size)

In [26]:
def strangeBlock_v4(input_block, nb_layers, filters):
    x_list = [input_block]
    c_temp = input_block
    for i in range(nb_layers):
        c_l = LSTM(units=filters, activation='elu', return_sequences=True) (c_temp)
        x_list.append(c_l)
        merge = Concatenate()(x_list)
        c_temp = merge
    return c_temp
    

In [27]:
def make_MLT_dense(in_shape, num_classes):
    input_tensor = Input(shape = in_shape)
    block = strangeBlock_v4(input_tensor, 5, 12)
    block_2 = strangeBlock_v4(block, 3, 20)
    landcover = LSTM(num_classes,activation='softmax', return_sequences=False, name='landcover') (block_2)
    canopy = LSTM(1, return_sequences=False, name='canopy') (block_2)
    model = Model(inputs=input_tensor, outputs=[landcover, canopy])
    return model

In [36]:
model = make_MLT_dense((4,7), class_count)
#model.load_weights('mlt_dense_model.hdf5')

In [9]:
#model = keras.models.load_model('dense_model.h5')

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [43]:
filepath = 'mlt_dense_model.hdf5'#your filepath here
checkpoint = ModelCheckpoint(filepath, monitor='val_landcover_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
tile_gen = rnn_tiles.rnn_tile_gen(landsat_datasets, lc_labels, canopy_labels, tile_size, class_count)

In [44]:
model.compile(optimizer=Adam(), loss={'landcover':'categorical_crossentropy', 'canopy':'mae'}, metrics={'landcover':['accuracy'], 'canopy':['mae']}, loss_weights={"landcover":1, "canopy":1})

In [45]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size, flatten=True, canopy=True), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                   validation_data=tile_gen.tile_generator(val_px, batch_size, flatten=True, canopy=True),
                  validation_steps=len(val_px) // batch_size, callbacks=callbacks_list)# class_weight=weights_list)# callbacks=callbacks_list) #class_weight=weights, callbacks=callbacks_list) 

Epoch 1/200
468/468 [==============================] - 58s 125ms/step - loss: 0.6644 - landcover_loss: 0.6057 - canopy_loss: 0.0587 - landcover_acc: 0.8033 - canopy_mean_absolute_error: 0.0587 - val_loss: 0.5991 - val_landcover_loss: 0.5420 - val_canopy_loss: 0.0572 - val_landcover_acc: 0.8209 - val_canopy_mean_absolute_error: 0.0572

Epoch 00001: val_landcover_acc improved from -inf to 0.82086, saving model to mlt_dense_model.hdf5
Epoch 2/200
468/468 [==============================] - 45s 96ms/step - loss: 0.6398 - landcover_loss: 0.5818 - canopy_loss: 0.0580 - landcover_acc: 0.8044 - canopy_mean_absolute_error: 0.0580 - val_loss: 0.5836 - val_landcover_loss: 0.5274 - val_canopy_loss: 0.0562 - val_landcover_acc: 0.8211 - val_canopy_mean_absolute_error: 0.0562

Epoch 00002: val_landcover_acc improved from 0.82086 to 0.82114, saving model to mlt_dense_model.hdf5
Epoch 3/200
468/468 [==============================] - 45s 95ms/step - loss: 0.6246 - landcover_loss: 0.5675 - canopy_loss: 0.

KeyboardInterrupt: 

In [ ]:
model.save('mlt_dense.h5')

# Evaluate model

In [46]:
predictions = model.predict_generator(generator = tile_gen.tile_generator(test_px, batch_size=1, flatten=True, canopy=True), steps=len(test_px) // 1, verbose=1)
eval_generator = tile_gen.tile_generator(test_px, batch_size=1, flatten=True, canopy=True)

2563/2563 [==============================] - 77s 30ms/step


In [ ]:
lc_predictions = np.asarray(predictions[0])
canopy_pred = np.asarray(predictions[1])
lc_predictions = np.argmax(lc_predictions,axis=-1)

In [ ]:
lc_lab = np.empty(lc_predictions.shape)
canopy_true = np.empty(canopy_pred.shape)
count = 0
while count < len(lc_predictions):
        image_b, label_b = next(eval_generator)
        #label_b = np.argmax(label_b, axis=-1)
        label_lc = np.argmax(label_b['landcover'], axis=-1)
        canopy_true[count] = label_b['canopy']
        lc_lab[count] = label_lc
        count += 1
label_index = lc_lab.reshape(len(test_px)*tile_size*tile_size)
pred_index = lc_predictions.reshape(len(test_px)*tile_size*tile_size)

In [ ]:
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
util.plot_confusion_matrix(label_index.astype(np.int), pred_index.astype(np.int), classes=np.array(list(util.indexed_dictionary)),
                      class_dict=util.indexed_dictionary)
# Plot normalized confusion matrix
util.plot_confusion_matrix(label_index.astype(np.int), pred_index.astype(np.int), classes=np.array(list(util.indexed_dictionary)),
                      class_dict=util.indexed_dictionary,
                      normalize=True)
count = 0
for i in range(len(label_index)):
    if(label_index[i] == pred_index[i]):
        count+=1
print("Accuracy is {}".format(count/len(label_index)))

In [ ]:
total = 0
tot = 0
for count in range(len(canopy_pred)):
    if canopy_true[count] != 0 and canopy_pred[count] !=0:
        if canopy_pred[count] < 0:
            canopy_pred[count] = 0
        total+= np.absolute(canopy_pred[count] - canopy_true[count])
        tot+=1
print(total/tot)

In [ ]:
plt.pyplot.hist(canopy_pred)

In [ ]:
plt.pyplot.hist(canopy_true)